GRADUATION

[algospot의 GRADUATION 링크](https://algospot.com/judge/problem/read/GRADUATION)  
  
### 문제  
1학년은 노는 게 남는 거란 선배의 말을 철석같이 믿고, 전공 과목은 다 수강철회하고 교양 과목은 다 F 받는 방탕한 1학년을 보냈던 태우는 이제 와서 자신의 행동을 후회하고 있습니다. 졸업 전에 채워야 할 학점이 너무 많기 때문입니다. 졸업 필수 학점을 채우려면 전공 과목 N 개 중 K 개 이상을 수강해야 합니다. 그런데 각 과목은 해당 과목의 선수과목을 미리 수강했어야만 수강할 수 있으며, 각 학기마다 모든 과목이 개설되는 것이 아니기 때문에 문제가 복잡해졌습니다. 어떻게 해야 최소 학기에 졸업을 할 수 있을까요?  
  
각 과목의 정보와 앞으로 M 학기 동안 개설될 과목의 목록이 주어질 때, 태우가 최소 몇 학기를 다녀야 졸업할 수 있는지 계산하는 프로그램을 작성하세요. 한 과목도 수강하지 않는 학기는 휴학한 것으로 하며, 다닌 학기 수에 포함되지 않습니다.  
  
### 출력  
한 줄에 태우가 다녀야 할 최소 학기 수를 출력합니다. M 학기 내에 졸업할 수 없는 경우 IMPOSSIBLE을 출력합니다.  

solution의  
majors_n : 전공 과목의 수 $(1 \leq$ majors_n $\leq 12)$   
required_n : 들어야할 과목의 수 $(0 \leq$ required_n $\leq $majors_n$)$   
semester_n : 학기의 수 $(1 \leq$ semester_n $\leq 10)$  
limit_n : 한학기에 들을 수 있는 최대 과목의 수 $(1 \leq$ limit_n $\leq 10)$  
prerequisites : 선수과목 $(1 \leq prerequisite_i \leq 10)$   
lectures : 학기당 열리는 과목들을 지니는 리스트 $(1 \leq lecture_i \leq 10)$   
  

In [1]:
from collections import Counter
import sys

majors_n = 0 # 전공 과목의 수
required_n = 0 # 들어야할 과목의 수
semester_n = 0 # 학기의 수
limit_n = 0 # 한 학기에 들을 수 있는 과목 수
mem = None # 메모이재이션

def graduate(semester, taken, prerequisites, lectures):
    global majors_n, required_n, semester_n, limit_n, mem
    c = Counter(bin(taken)[2:])
    if c['1'] >= required_n : # 이미 들은 과목 수가 들어야할 과목 수 보다 클 때(base case)
        return 0 
    if semester == semester_n: # 만약 모든 학기를 모두 탐색한 경우(base case)
        return sys.maxsize # 주어진 학기 내에 졸업 불가
    if mem[semester][taken] != -1: # 만약 이미 계산해놓은 상태일 경우
        return mem[semester][taken] # 메모이재이션
    
    mem[semester][taken] = sys.maxsize # 현재 학기와 상황에 대한 상태
    canTake = (~taken) & lectures[semester]
    # 현재 들을 수 있는 과목 비트마스크 = (듣지 않은 과목) ∩ (이번 학기 개설된 과목)
    
    for i in range(len(prerequisites)): # 모든 과목에 대해서
        if (canTake & (1 << i)) and (prerequisites[i] & taken) != prerequisites[i]:
        # (들을 수 있는 i번째 과목) ∩ (i번째 과목에 대한 선수과목 중 이미 들은 과목)
        # 위의 연산이 i번째 과목의 선수과목과 다를 때
            canTake &= ~(1 << i)
            # i번째 과목은 들을 수 없으므로 들을 수 있는 과목에서 제외
    
    take = canTake
    while take > 0:
        c = Counter(bin(take)[2:])
        if c['1'] > limit_n: # 만약 들을 과목이 한 한기 최대 과목 수를 초과할 경우 
            take = ((take - 1) & canTake)
            continue # 다음 턴으로 넘어감
        mem[semester][taken] = min(mem[semester][taken], graduate(semester+1, taken | take, prerequisites, lectures) + 1)
        # 재귀 (이번 학기에 들을 수 있는 과목을 들은 후, 다음 학기에 대해서 다시 탐색)
        take = ((take - 1) & canTake)
        # take = canTake의 부분집합
    # 즉, 들을 수 있는 과목들의 모든 부분집합(공집합 제외)에 대해서 탐색    
    mem[semester][taken] = min(mem[semester][taken], graduate(semester+1, taken, prerequisites, lectures))
    # 그리고 마지막으로 공집합에 대해서 탐색
    return mem[semester][taken]
    
def solution(m_n, r_n, s_n, l_n, prerequisites, lectures):
    global majors_n, required_n, semester_n, limit_n, mem
    
    majors_n = m_n 
    required_n = r_n
    semester_n = s_n
    limit_n = l_n
    mem = [[-1 for _ in range(1 << majors_n)] for _ in range(semester_n)]
    # 초기화
    
    answer = 0
    bit_prerequisites = [] # 선수과목에 대한 비트마스크
    bit_lectures = [] # 학기당 열리는 과목에 대한 비트마스크
    
    for i in prerequisites:
        bit = 0
        for j in range(len(i)):
            bit += 1 << i[j]
        bit_prerequisites.append(bit)
    
    for i in lectures:
        bit = 0
        for j in range(len(i)):
            bit += 1 << i[j]
        bit_lectures.append(bit)
    # 비트마스크 초기화
        
    answer = graduate(0, 0, bit_prerequisites, bit_lectures)
    # 처음 0번째 학기, 아무것도 수강하지 않은 상태에서 부터 탐색
    
    if answer == sys.maxsize: # 정해진 학기 내 졸업이 불가능한 경우
        return "IMPOSSIBLE"
    return answer # 가능하다면 답 반환
    

In [2]:
majors_n = 4
required_n = 4
semester_n = 4
limit_n = 4
prerequisites = [[],[0],[0, 1, 3], []]
lectures = [[0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 3], [0, 1, 2, 3]]

In [3]:
solution(majors_n, required_n, semester_n, limit_n, prerequisites, lectures)

3

In [4]:
majors_n = 4
required_n = 2
semester_n = 2
limit_n = 4
prerequisites = [[1],[],[3],[2]]
lectures = [[0,2,3],[1,2,3]]

In [5]:
solution(majors_n, required_n, semester_n, limit_n, prerequisites, lectures)

'IMPOSSIBLE'